# 第2回：決定木から始める機械学習

このHands-onでは，機械学習手法のひとつである**決定木**を使って，あらかじめ与えられたデータから，未知データを分類する規則を抽出・適用する**教師あり学習**を体験します．この演習で用いるデータは以下の通りです：

* アヤメ（花の種類）のデータ
* タイタニック号の乗船者データ

演習に先立って，必要なライブラリを準備します．まず，Google Colaboratoryに
* graphviz
* category_encoders

の2つのライブラリをインストールするために， 以下のコードをGoogle Colaboratoryで実行します．

In [ ]:
try:
    import category_encoders
    import graphviz
except:
    !pip install graphviz
    !pip install category_encoders

続けて，必要なライブラリを読み込みます．以下のコードを実行してください．

In [ ]:
# 表形式のデータを操作するためのライブラリ
import pandas as pd

# 行列計算をおこなうためのライブラリ
import numpy as np

# 機械学習用ライブラリsklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz

# その他
import category_encoders

# グラフ描画ライブラリ
from graphviz import Source
import matplotlib.pyplot as plt
%matplotlib inline


---
## 例題1: アヤメ

データマイニングや機械学習を学ぶ際，例題データとしてアヤメ（英語名:Iris）データがよく用いられます（[アヤメ](https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%A4%E3%83%A1)は植物の1つです）． 決定木アルゴリズムを体験する題材として，このHands-onでもアヤメデータを使ってみましょう．

以下のコードを実行して，アヤメのデータを読み込みます．

In [ ]:
from sklearn import datasets

# Iris（アヤメ）の大きさに関するデータをロード
iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['species'] = iris.target_names[iris.target]

# 簡単のために，カラム名を修正しておく
iris_df = iris_df.rename(
    columns = {
        'sepal length (cm)': 'sepal_length',
        'sepal width (cm)': 'sepal_width',
        'petal length (cm)': 'petal_length',
        'petal width (cm)': 'petal_width'
    }
)

# 最初の数件を表示
iris_df.head()

このアヤメデータには，花弁（petal）の長さ・幅，がく（sepal）の長さ・幅，品種が記されています．例題1の目標は，**花弁の長さ・幅，がくの長さ・幅から品種を推定する予測モデルを構築**することです．早速，決定木を用いて予測モデルを構築してみましょう．

一般に教師あり学習で予測を行うモデルを構築する際には，データを**学習用（訓練）データ**と**評価用データ**に分割してデータ分析を行います．以下のコードを実行して，先ほど用意したデータを学習用（70%）と評価用（30%）に分割します．

In [ ]:
# データを学習用（70%）と評価用（30%）に分割する
iris_train_df, iris_test_df = train_test_split(
                                iris_df, test_size=0.3,
                                random_state=1,
                                stratify=iris_df.species)

変数``iris_test_df``には品種情報も含まれますが，予測モデルの性能評価の際には，品種情報が未知であるとして予測を行い，予測結果と（隠しておいた）品種情報を照らし合わせて評価することになります．

では，教師あり学習のひとつである決定木アルゴリズムを適用してみましょう．``iris_train_df``に決定木アルゴリズムを適用して，品種を見分けるルールを抽出（学習）しましょう．決定木アルゴリズムは`sklearn`ライブラリの``DecisionTreeClassifier``クラスを使って実行できます．下記コードを実行してみてください．

In [ ]:
# X_trainは，品種（Species）以外のすべての指標
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
X_train = iris_train_df[features]

# y_trainは品種の指標
y_train = iris_train_df.species

# 学習
model = DecisionTreeClassifier(criterion='entropy',
                               random_state=12345) # 初期値を固定
model.fit(X_train, y_train)

品種を予測するルールが学習されました．予測ルールをわかりやすく可視化してみましょう．以下のコードを実行してみてください．

In [ ]:
Source(export_graphviz(model, out_file=None,
                       feature_names=features,
                       class_names=['setosa', 'versicolor', 'virginica'],
                       proportion=True,
                       filled=True, rounded=True # 見た目の調整
                      ))

分類ルールが木のように枝分かれした形で可視化されました．この可視化結果が，今回の教師あり学習アルゴリズムが **決定「木」** と呼ばれる所以です．

結果の見方ですが，各四角が分類ルールの分岐をあわらしています．四角の下に書かれた文字情報が分岐条件を示しています．四角中に書かれた文字は，四角に至るまでに適用された分岐条件を満たすと，
* その条件を満たすデータが全体の何パーセントあるか
* ラベルごとの分類結果の割合が何パーセントか

を示しています．例えば，上図の上から3段目の左にある「class=versicolor, value=\[0.0, 1.00, 0.0\]」という四角は，
* 花弁（petal）の長さが2.6より大きい，かつ花弁（petal）の長さが4.75以下の場合，その個体は100%の確率でversicolorであること
* この条件にマッチする個体はデータセットに28.6%存在すること

を示しています．

さて，ここまでやったことは予測のためのルール（モデル）の構築でした．構築した予測モデルを使って，未知のデータを予測してみましょう．この例題の冒頭で，変数``iris_test_df``に**予測モデルの構築に使われていないデータ**を別途用意していたことを思い出しましょう．

In [ ]:
# 最初の数件を表示
iris_test_df.head()

先ほど構築した予測モデルをこの``iris_test_df``に適用して，未知データのアヤメの品種を予測してみましょう．構築した予測モデル``iris_model``を用いて未知データを予測するには``predict``関数を用います．

In [ ]:
# 評価用データの特徴量と正解ラベルを取得
X_test = iris_test_df[features]
y_test = iris_test_df.species

# 予測モデルを使って，品種が未知の個体の品種を推定
iris_predicted = model.predict(X_test)

# 予測結果の一部を表示
iris_predicted

予測結果が変数``iris_predicted``に格納されました．``iris_test_df``の列``Species``には実際の品種情報が格納されていました．これと予測結果と照らし合わせて，予測性能を評価してみましょう．

予測性能の評価指標には様々なものがありますが，ここでは精度（accuracy）を計算しましょう．精度は「予測結果のうち， **各個体の品種について，予測モデルが予測したものと，実際の品種が一致したケースの割合」** を意味します．精度の計算には`sklearn`の`accuracy_score`関数を用います．第1引数に予測結果，第2引数に実際の結果を入力します．以下のコードを実行してみましょう．

In [ ]:
accuracy_score(iris_predicted, iris_test_df.species)

上記結果によると，Accuracyは約97.8%を示しており，かなりの精度で品種を予測できていることが分かります．


---
## 例題2: タイタニック号の乗船者データ

1912年4月14日，処女航海中の豪華客船タイタニック号は多くの乗船者を乗せたまま沈没しました．タイタニックとその事故は，映画化されるなどして世界的に有名です．
乗船者に関する情報が残っていたために，事故後，多くの人が事故に関する分析を行いました．私たちもタイタニック号の乗船者情報を用いて，生死を分けた条件について分析を行ってみましょう．

以下のコードを実行して，タイタニック号の乗船者（の一部）のデータを読み込みましょう．

In [ ]:
# データの読み込み
url = "https://raw.githubusercontent.com/hontolab-courses/dmml-2022/main/dataset/titanic_train.csv"
titanic_df = pd.read_table(url, header=0, sep=",")

# 生存情報を分かりやすくする
titanic_df = titanic_df.assign(
    Survived = lambda df: df.Survived.map({1: 'survived', 0: 'died'})
)

# 最初の数件のみ表示
titanic_df.head()

様々な情報が表示されました．変数``titanic_train_df``に格納されたデータの属性（列名）の詳細は以下の通りです：

* PassengerId: 乗船者を識別するためのID
* Survived: ある乗船者が沈没事故で生き残った否かを示すフラグ．
* Pclass: チケットの等級．1は1等乗客，2は2等乗客，3は3等乗客を表す
* Name: 乗客名
* Sex: 性別
* Age: 年齢
* SibSp: タイタニック号に同乗した兄弟もしくは配偶者の数
* Parch: タイタニック号に乗船した両親もしくは子どもの数
* Ticket: チケット番号
* Fare: 乗船料金
* Cabin: 客室番号
* Embarked: 乗船した港．C = Cherbourg, Q = Queenstown, S = Southampton

このデータを用いて，どんな乗客が生き残れたのかを予測できるようにしましょう．

決定木を適用する前に，``titanic_df``データに対して簡易的な分析を行い，各データ属性と生存情報との関係を眺めてみましょう．
以下のコードを実行すると， **乗客の等級（Pclass）と生存の有無（Survived）** の属性の値を集計して，ある等級の乗客のうち生き残った方の割合が表示されます．

In [ ]:
pd.crosstab(titanic_df['Survived'], titanic_df['Pclass'], normalize='columns')

分析の結果，どうやら等級が高い（数値が小さい）ほど生き残っている方の割合が大きいようです．等級以外の属性でも同様の分析を行ってみてください．例えば，性別（Sex）と生存の有無の関係は以下のコードで得られます．

In [ ]:
pd.crosstab(titanic_df['Survived'], titanic_df['Sex'], normalize='columns')

決定木アルゴリズムを適用する前に，データの欠損を確認しておきます．
収集したデータの一部が欠損していることはよくあることです．欠損値がデータに含まれると，機械学習のアルゴリズムがうまく動作しない場合があります．

欠損値がある場合の対応は，
* 欠損しているデータを捨てる
* 欠損値を代表的な値で埋める

といったアプローチが採られることが多いです．欠損しているデータを捨ててしまうと，学習に用いる貴重なデータが減りますので，今回は欠損値を代表値で埋めます．

まず，以下のコードを走らせて，欠損値を確認してみましょう．

In [ ]:
titanic_df.isnull().sum()

上の結果から，Age，Cabin，Embarkedに欠損値が含まれるようです．Cabinは乗船客に与えられた固有の情報で，生存者の予測には役立ちません．それゆえ，AgeとEmbarkedのみ欠損値を埋めることにします．

欠損値を埋めるには様々な方法が提案されていますが，今回は
* Ageは中央値
* Embarkedは最頻値

で埋めることにします．以下のコードを実行してください．

In [ ]:
# Embarkedの欠損を最頻値で埋める
titanic_df["Embarked"] = titanic_df["Embarked"].fillna(titanic_df["Embarked"].mode().iloc[0]) 

# Ageを中央値で埋める
titanic_df["Age"] = titanic_df["Age"].fillna(titanic_df["Age"].median()) 

これで欠損値はなくなりました．
それでは決定木アルゴリズムを適用してみましょう．例題1と同様，まず，用意したデータを学習用（70%）と評価用（30%）に分割します．

In [ ]:
# データを学習用（70%）と評価用（30%）に分割する
titanic_train_df, titanic_test_df = train_test_split(
                                        titanic_df, test_size=0.3,
                                        random_state=1,
                                        stratify=titanic_df.Survived)

変数``titanic_test_df``には生存の有無の情報も含まれますが，予測モデルの性能評価の際には，生存情報が未知であるとして予測を行い，予測結果と（隠しておいた）生存情報を照らし合わせて評価することになります．

簡易的な分析を行ってみると，生存の有無を識別するために有効な指標がありそうな気もします．しかし実際には，複数の指標が絡み合って生存の有無が決まっていると思われます．このような状況で，指標（特徴量）同士の複雑な関係性を考慮しながら，予測のためのルールを抽出するのが**教師あり学習**です．

早速，決定木アルゴリズムを適用してみましょう．まずは決定木を適用するデータを整形します．データを眺めると，氏名（Name）やチケット番号（Ticket），客室番号（Cabin）は各乗船者に固有に与えられた情報であることが分かります．これら特徴量は生存者の予測には役に立たないため，それ以外の情報を利用することにします．

下記コードを実行して，決定木を適用する際に注目する指標を，変数``target_features``に格納しておきます．さらに，
``titanic_train_df``から上記指標に関するデータのみを抽出します．

In [ ]:
# 注目する指標
target_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

# 以下のように書けば，target_featuresの指標のみに注目してデータを抽出できる
titanic_train_df[target_features]

性別（Sex）や乗船した港（Embarked）は数値情報ではなくカテゴリ情報です．多くの機械学習は数値を受け取って処理をするので，カテゴリ情報も数値情報に変換しておいた方が都合がよいです．ここでは，「EmbarkedがSであることをEmbarked_Sが1，EmbarkedがSでないことをEmbarked_S=0」となるような変換をおこなうことにします．

下記コードがその準備となります．

In [ ]:
encoder = category_encoders.OneHotEncoder(cols=['Embarked', 'Sex'], use_cat_names=True)
encoder.fit(titanic_train_df[target_features])

それでは，``titanic_train_df``に決定木アルゴリズムを適用して，生存の有無のルールを抽出（学習）しましょう．決定木アルゴリズムは``DecisionTreeClassifier``クラスを用いて実行できます．下記コードを実行してみてください．

In [ ]:
# 予測に用いる生存情報以外のすべての指標をX_trainに
X_train = titanic_train_df[target_features]

# カテゴリ変数を数値情報に変換
X_train = encoder.transform(X_train)

# y_trainは生存有無をあらわす指標
y_train = titanic_train_df.Survived

# 学習
model = DecisionTreeClassifier(criterion='entropy',
                               random_state=12345, # 初期値を固定
                               max_depth=3) # 木の深さを3に限定
model.fit(X_train, y_train)

生存の有無を予測するルールが学習されました．生存の有無を予測するためのルールをわかりやすく可視化してみましょう．以下のコードを実行してみてください．

In [ ]:
Source(export_graphviz(model, out_file=None,
                       feature_names=X_train.columns,
                       class_names=['died', 'survived'],
                       proportion=True,
                       filled=True, rounded=True # 見た目の調整
                      ))

分類ルールが得られました．

結果を解釈してみましょう．例えば，上図の上から3段目，左端にある「class=survived, entropy=0.258」という四角は，
* 性別が女性であり（Sex_male<=0.5: True），乗船クラスが1等もしくは2等クラス（Pclass<=2.5: True）の乗客は95.7%の確率で生存したこと
* その条件にマッチする乗客は，全体の18.5%存在すること

を示しています．

なんとなく予測ルールは分かりましたが，各指標が予測にどの程度影響があるかを調べてみましょう．以下のコードを実行します．

In [ ]:
for feature, importance in zip(X_train.columns, model.feature_importances_):
    print("{}\t{}".format(feature, importance))

この結果からも，**性別**や**等級**が生存に大きく影響を与えていたことがうかがえます．

さて，ここまでやったことは予測のためのルール（モデル）の構築でした．構築した予測モデルを使って，未知のデータを予測してみましょう．この例題の冒頭で，変数``titanic_test_df``に**予測モデルの構築に使われていないデータ**を別途用意していたことを思い出しましょう．

In [ ]:
# 最初の数件を表示
titanic_test_df.head()

先ほど構築した予測モデルをこの``titanic_test_df``に適用して，生存の有無を予測してみましょう．構築した予測モデル``model``を用いて未知データを予測するには``predict``メソッドを用います．

In [ ]:
# X_testは，生存情報以外のすべての指標
X_test = titanic_test_df[target_features]

# カテゴリ変数を計算しやすく変換する
X_test = encoder.transform(X_test)

# 予測
y_predicted = model.predict(X_test)

# 予測結果（最初の10件）
y_predicted[:10]

予測結果が変数``y_predicted``に格納されました．``titanic_test_df``の列``Survived``には実際の生存情報が格納されていました．これと予測結果と照らし合わせて，予測性能を評価してみましょう．以下のコードを実行して，予測性能の評価を行います．

In [ ]:
# y_testは生存の指標
y_test = titanic_test_df.Survived

accuracy_score(y_predicted, y_test)

色々情報が出てきましたが，``Accuracy``という数値を見てください．Accuracyは予測結果のうち，
**実際に生存した乗客を予測モデルが「生存」と予測し，死亡した乗客を予測モデルが「死亡」と予測できたケースの割合**を示しています．

上記結果によると，Accuracyは約76.1%を示しており，そこそこの割合で生存の有無を予測できていることが分かります．



---

## 演習課題

以下のコードを実行して`income_df`に格納されるデータは，ある年にアメリカで実施された国勢調査のデータである．

In [ ]:
# データの読み込み
income_df = pd.read_table("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", sep=',', header=None)

# 列名（特徴）に名前を付ける
income_df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 
                     'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

# データ表示（先頭5件）
income_df.head()

データ中の列名（特徴量）の意味は以下の通りである：

* age: 年齢（整数）
* workclass: 雇用形態（公務員，会社員など）
* fnlwgt: 使わない
* education: 学歴
* education-num: 使わない
* marital-status: 婚姻状態
* occupation: 職業
* relationship: 家族内における役割
* race: 人種
* sex: 性別
* capital-gain: 使わない
* capital-loss: 使わない
* hours-per-week: 週あたりの労働時間（整数値）
* native-country: 出身国
* income: 年収（50Kドル以上，50Kドル未満の二値）

このデータに対して決定木アルゴリズムを適用して，ある人物が年間収入が50Kドル以上か未満かを分類する機械学習モデルを構築したい．

### 課題1
機械学習モデルを構築する前に，基礎データとして`income_df`データに含まれる調査対象者の年齢，性別，年収の分布を知りたい．
年齢に関するヒストグラム（階級数は10）を作成せよ．また，性別（男，女），年収（50K以上，50K未満）について，属性値に対応する人数を求めよ．

* ヒント1: ヒストグラムの作成には`pandas.series.hist`関数を用いるとよい（[参考](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.hist.html)）
* ヒント2: 要素の出現頻度を求めるには`pandas.series.value_counts`メソッドを用いるとよい（[参考](https://note.nkmk.me/python-pandas-value-counts/)）

### 課題2
``income_df``データを集約し，学歴ごとに年間収入クラスの内訳（割合）を調べよ．

* ヒント3: pandasのcrosstabメソッドを使う（タイタニックの例を再訪すること）

### 課題3
`income_df`データに決定木アルゴリズムを適用し，年収（`income`）の分類における各属性（列）の寄与度を求めよ．

### 課題4
課題3の結果をもとに年収分類に寄与する特徴量を（最大5つ）特定し，その特徴量のみを用いて再度決定木モデルを構築せよ．その際，できる限りシンプルなモデルになるよう，あまり木が深くならないよう調整すること．

### 課題5
課題4で構築した決定木を可視化し，年収の多寡に影響を与える条件について考察せよ．